Introduction to Natural Language Processing
* Getting the necessary doc and files

In [ ]:
# !wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
# !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

--2024-12-11 20:30:43--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.212.123, 142.250.200.251, 172.217.18.59, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.212.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: 'nlp_getting_started.zip'

     0K .......... .......... .......... .......... ..........  8%  145K 4s
    50K .......... .......... .......... .......... .......... 16%  280K 3s
   100K .......... .......... .......... .......... .......... 25%  531K 2s
   150K .......... .......... .......... .......... .......... 33%  559K 1s
   200K .......... .......... .......... .......... .......... 42%  721K 1s
   250K .......... .......... .......... .......... .......... 50%  667K 1s
   300K .......... .......... .......... .......... .......... 59%  381K 1s
   350K .......... .......... ..........

In [3]:
from helper_functions import unzip_data, plot_loss_curves, create_tensorboard_callback, compare_historys
unzip_data('nlp_getting_started.zip')

In [1]:
import pandas as pd
import numpy as np

train_data = pd.read_csv(r'train.csv')
test_data = pd.read_csv(r'test.csv')
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [2]:
# shuffling the train_data
train_data = train_data.sample(frac=1, random_state=42)
train_data.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [3]:
train_data.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [4]:
len(train_data), len(test_data)

(7613, 3263)

In [5]:
import random
random_index = random.randint(0, len(train_data)-5)

for row in train_data[['text', 'target']][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target :{target} ", "(real diaster)" if target>0 else "(not real diseater)")
    print(f"Text :{text} \n")
    print('......\n')

Target :0  (not real diseater)
Text :@stupid_niggr I'm telling your mom your up past curfew oth 

......

Target :1  (real diaster)
Text :Japan marks 70th anniversary of Hiroshima atomic bombing http://t.co/a2SS7pr4gW 

......

Target :1  (real diaster)
Text :Firefighters Headed To California To Fight Wild Fires http://t.co/J2PYkYo0EN 

......

Target :1  (real diaster)
Text :Urgent! Save the Salt River #WildHorses! Mass murder by the very ppl supposed to protect them?  --&gt; http://t.co/14wH0pJJ2C @CNN @CBC 

......

Target :0  (not real diseater)
Text :#SBNation #Flames What Makes a Good Penalty Killer? http://t.co/xYi5fDacxO http://t.co/SjtvzgGcXU 

......



Spliting data into training, testing anad validation sets

In [6]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_data['text'].to_numpy(), 
                                                                            train_data['target'].to_numpy(),
                                                                            test_size=0.1, 
                                                                            random_state=42) 

In [7]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [8]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

**converting text to numbers**
* Tokenization: direct mapping of tekens(a token could be a word or a character)
* Embedding: creating a metrix of feature vectors for each token(s)(the size of the feature vector can be define and the embedding can be learned)

In [9]:
# Tokenization
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [10]:
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize = 'lower_and_strip_punctuation',
                                    split = 'whitespace',
                                    ngrams= None,
                                    output_mode = 'int',
                                    output_sequence_length = None,
                                    pad_to_max_tokens = False)

In [11]:
len(train_sentences[0].split())

7

In [12]:
# findinf the number of avg word in ine training sentences
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [13]:
# setting up text vectorizer
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    output_mode = 'int',
    output_sequence_length = max_length
)

In [14]:
text_vectorizer.adapt(train_sentences)

In [15]:
# creating a smaple sentence tokens

sentence = "This is a sample sentence statement"
text_vectorizer([sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  19,    9,    3, 8839,    1, 2283,    0,    0,    0,    0,    0,
           0,    0,    0,    0]], dtype=int64)>

In [16]:
# choosing the random sentence in the training set
random_sentence = random.choice(train_sentences)
print(f"Origin text: \n{random_sentence}\n")

text_vectorizer([random_sentence])

Origin text: 
#GrowingupBlack walking past chicken frying was like entering a war zone.



<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[5533, 1239,  501, 4013,    1,   23,   25, 3884,    3,  122,  471,
           0,    0,    0,    0]], dtype=int64)>

In [17]:
# getting the unique characters in the vocabulary
word_in_vocab = text_vectorizer.get_vocabulary()
top_5 = word_in_vocab[:5]
buttom_5 = word_in_vocab[-5:]
print(len(word_in_vocab))
print(top_5)
print(buttom_5)

10000
['', '[UNK]', 'the', 'a', 'in']
['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [18]:
# Setting the embedding space 
from tensorflow.keras.layers import Embedding
embedding = Embedding(
    input_dim = max_vocab_length,
    input_length = max_length,
    output_dim = 128
)

c:\Users\Phavour\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [19]:
# getting random number of embedding
print(f"Original sentences :\n{random_sentence}\n")
print(f"Embedded sentences : \n")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original sentences :
#GrowingupBlack walking past chicken frying was like entering a war zone.

Embedded sentences : 



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00162905, -0.02688622, -0.03022641, ...,  0.02558925,
         -0.03335335, -0.02169587],
        [-0.0384636 , -0.01635982,  0.0114094 , ...,  0.02910955,
         -0.04642415,  0.04390479],
        [ 0.01998502,  0.00738205, -0.00874287, ..., -0.01295625,
         -0.0172757 , -0.01607299],
        ...,
        [-0.04774345,  0.00279681,  0.01735611, ...,  0.01403593,
          0.03249322, -0.03701786],
        [-0.04774345,  0.00279681,  0.01735611, ...,  0.01403593,
          0.03249322, -0.03701786],
        [-0.04774345,  0.00279681,  0.01735611, ...,  0.01403593,
          0.03249322, -0.03701786]]], dtype=float32)>

In [20]:
# checking out a single token
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.00162905, -0.02688622, -0.03022641,  0.00019073,  0.04606848,
       -0.03169955,  0.02028806, -0.0399112 , -0.04113709, -0.00198804,
        0.01215966,  0.04118789,  0.01400055,  0.03401591,  0.00753906,
       -0.01327474, -0.04175868, -0.01185417,  0.00762756,  0.01410191,
       -0.03254277, -0.04778935, -0.044621  , -0.04360216, -0.00011094,
        0.02581784,  0.01851198,  0.02191367, -0.02309622, -0.03286263,
       -0.01954057,  0.03715907,  0.00071945,  0.00453655, -0.04535774,
        0.01499769,  0.01502166,  0.02736959, -0.02194239, -0.04165783,
        0.024609  ,  0.02167049,  0.00775517,  0.04735562,  0.00402459,
       -0.00013862,  0.01773742,  0.00678941, -0.00946299,  0.02591891,
       -0.02440034,  0.04907897, -0.01245471,  0.03573779,  0.0420669 ,
       -0.01195247,  0.03738577,  0.029334  ,  0.02621355, -0.04059043,
        0.01473508, -0.01861892,  0.01521379,  0.00631632, -0.04552039,
       -0.033359

Modelling text datase(running a series of expperiment)
* First Baseline

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [22]:

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# fitting the model
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [23]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Model 0 score: {baseline_score*100:.2f}")

Model 0 score: 79.27


In [24]:
baseline_predictions = model_0.predict(val_sentences)
baseline_predictions[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0], dtype=int64)

In [25]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int64)

**creating a function to keep track of our model accuracies**
* F1 scores
* precision
* accuracy
* recall


In [26]:
# from sklearn.metrics import accuracy_score, recall, f1_score, precision
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_accuracy(y_true, y_pred):
    model_accuracy = accuracy_score(y_true, y_pred)*100
    model_precision, model_recall, model_f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    model_results = {
        "model accuracy": model_accuracy,
        "model precision": model_precision*100,
        "model recall": model_recall*100,
        "model f1 score": model_f1_score*100
    }

    return model_results



In [27]:
baseline_results = calculate_accuracy(val_labels, baseline_predictions)
baseline_results

{'model accuracy': 79.26509186351706,
 'model precision': 81.11390004213173,
 'model recall': 79.26509186351706,
 'model f1 score': 78.6218975804955}

model 2 Dense model(Feed forward Neural Network)

In [28]:
logs_dir = 'model logs'

In [29]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs, output, name='model_2')

model_2.summary()

Model: "model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
from helper_functions import create_tensorboard_callback
model_2.compile(
    loss = 'binary_crossentropy',
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)



In [31]:
train_sentences.shape, train_labels.shape, val_sentences.shape, val_labels.shape

((6851,), (6851,), (762,), (762,))

In [32]:
model_2_history = model_2.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(logs_dir, experiment_name = 'model_2')]
)

Saving TensorBoard log files to: model logs/model_2/20241214-174222
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.6535 - loss: 0.6483 - val_accuracy: 0.7664 - val_loss: 0.5361
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.8103 - loss: 0.4623 - val_accuracy: 0.7835 - val_loss: 0.4711
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.8585 - loss: 0.3565 - val_accuracy: 0.7861 - val_loss: 0.4611
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.8895 - loss: 0.2896 - val_accuracy: 0.7861 - val_loss: 0.4638
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9116 - loss: 0.2421 - val_accuracy: 0.7940 - val_loss: 0.4871


In [33]:
model_2.evaluate(val_sentences, val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7818 - loss: 0.5222


[0.4870931804180145, 0.7939632534980774]

In [34]:
model_2_pred_prop = model_2.predict(val_sentences)
model_2_pred_prop.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step


(762, 1)

In [35]:
model_2_pred_prop[:10]

array([[0.33117652],
       [0.7647568 ],
       [0.9973897 ],
       [0.08659207],
       [0.11668901],
       [0.9224322 ],
       [0.9027504 ],
       [0.99189824],
       [0.95044124],
       [0.18445897]], dtype=float32)

In [36]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int64)

In [37]:
# model prediction propability to labels
model_pred = tf.squeeze(tf.round(model_2_pred_prop))
model_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [38]:
model_2_results = calculate_accuracy(val_labels, model_pred)
model_2_results

{'model accuracy': 79.39632545931758,
 'model precision': 80.31272377768599,
 'model recall': 79.39632545931758,
 'model f1 score': 78.99134672804105}

In [39]:
import numpy as np
np.array(list(model_2_results.values())) > np.array(list(baseline_results.values()))

array([ True, False,  True,  True])

Visualizing the embedding

In [49]:
word_in_vocab = text_vectorizer.get_vocabulary()
print(len(word_in_vocab))
word_in_vocab[-5:]

10000


['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

In [54]:
embed_weights = model_2.get_layer('embedding').get_weights()[0]
embed_weights

array([[-0.01886771,  0.03080338,  0.00612854, ..., -0.0143519 ,
         0.00438248, -0.00994216],
       [ 0.01597167, -0.0357241 ,  0.04235323, ..., -0.01135255,
        -0.05187606,  0.05865624],
       [-0.00572774,  0.0650195 , -0.03403732, ...,  0.01241109,
         0.00131982,  0.05828769],
       ...,
       [-0.00050465,  0.00988654, -0.02604703, ..., -0.04565628,
        -0.04242872, -0.04938591],
       [ 0.01114854,  0.00395703, -0.06694023, ..., -0.03059811,
         0.00599368,  0.01928031],
       [ 0.0411145 ,  0.04594398, -0.09519627, ..., -0.03251867,
        -0.06597479,  0.08369581]], dtype=float32)

In [55]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(word_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

model 3(Recurrent neural networks)

In [58]:
inputs = tf.keras.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.LSTM(64, return_sequences=True)(x) // for stacking multiple rrn(S)
x = layers.LSTM(64)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs,name="model_3")
model_3.summary()

Model: "model_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,329,473 (5.07 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [59]:
# compiling the model
model_3.compile(
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

In [60]:
# fittng the model
model_3.fit(
    train_sentences,
    train_labels,
    epochs= 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(logs_dir, experiment_name='model_3')]
)

Saving TensorBoard log files to: model logs/model_3/20241214-211237
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 33s 64ms/step - accuracy: 0.8805 - loss: 0.3072 - val_accuracy: 0.7795 - val_loss: 0.5192
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - accuracy: 0.9408 - loss: 0.1570 - val_accuracy: 0.7782 - val_loss: 0.6026
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 53ms/step - accuracy: 0.9492 - loss: 0.1302 - val_accuracy: 0.7808 - val_loss: 0.7489
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - accuracy: 0.9594 - loss: 0.1025 - val_accuracy: 0.7769 - val_loss: 0.8449
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 21s 54ms/step - accuracy: 0.9646 - loss: 0.0877 - val_accuracy: 0.7769 - val_loss: 0.8667


In [62]:
model_3_pred_prop = model_3.predict(val_sentences)
model_3_pred = tf.squeeze(tf.round(model_3_pred_prop))
print(model_3_pred[:10])
print(val_labels[:10])

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
tf.Tensor([0. 1. 1. 0. 0. 1. 1. 1. 1. 1.], shape=(10,), dtype=float32)
[0 0 1 1 1 1 1 1 1 0]


In [64]:
calculate_accuracy(val_labels, model_3_pred)

{'model accuracy': 77.69028871391076,
 'model precision': 77.77598953427996,
 'model recall': 77.69028871391076,
 'model f1 score': 77.53556135933601}

Bulding a Gated Recurrent Neural Network(GRU) model

In [72]:
inputs = tf.keras.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# x = layers.LSTM(32, return_sequences=True)(x)
# x = layers.GRU(16, return_sequences=True)(x)
# x = layers.GRU(16)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name = 'model_4')

model_4.summary()

Model: "model_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)     │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_17 (GRU)                    │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,317,313 (5.03 MB)

 Trainable params: 1,317,313 (5.03 MB)

 Non-trainable params: 0 (0.00 B)

In [73]:
model_4.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [74]:
model_4_history = model_4.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(logs_dir, experiment_name ='model_4')]
)

Saving TensorBoard log files to: model logs/model_4/20241214-214735
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 24s 34ms/step - accuracy: 0.8917 - loss: 0.2579 - val_accuracy: 0.7808 - val_loss: 0.7758
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - accuracy: 0.9716 - loss: 0.0772 - val_accuracy: 0.7730 - val_loss: 0.9058
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9727 - loss: 0.0690 - val_accuracy: 0.7730 - val_loss: 0.9184
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.9758 - loss: 0.0595 - val_accuracy: 0.7717 - val_loss: 1.1591
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9790 - loss: 0.0495 - val_accuracy: 0.7782 - val_loss: 1.3062


In [78]:
model_4_pred_prop=  model_4.predict(val_sentences)
model_4_pred=  tf.squeeze(tf.round(model_4_pred_prop))
model_4_pred[:5]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

In [79]:
calculate_accuracy(val_labels, model_4_pred)

{'model accuracy': 77.82152230971128,
 'model precision': 77.96479981380524,
 'model recall': 77.82152230971128,
 'model f1 score': 77.63875633348853}

In [80]:
import numpy as np
np.array(list(calculate_accuracy(val_labels, model_4_pred).values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

Building a bidirectional RNN

In [82]:
from tensorflow.keras import layers

inputs_5 = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs_5)
x = embedding(x)

x = layers.Bidirectional(layers.LSTM(64))(x)
outputs_5 = layers.Dense(1, activation='sigmoid')(x)
model_5 = tf.keras.Model(inputs_5, outputs_5, name='model_5')

In [83]:
model_5.summary()

Model: "model_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_13 (InputLayer)     │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,378,945 (5.26 MB)

 Trainable params: 1,378,945 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

In [84]:
model_5.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [85]:
model_5.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(logs_dir, experiment_name ='model_5')]
)

Saving TensorBoard log files to: model logs/model_5/20241214-222953
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 15s 34ms/step - accuracy: 0.9394 - loss: 0.1999 - val_accuracy: 0.7612 - val_loss: 0.8851
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9784 - loss: 0.0524 - val_accuracy: 0.7717 - val_loss: 1.1214
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9797 - loss: 0.0430 - val_accuracy: 0.7572 - val_loss: 1.1173
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.9815 - loss: 0.0435 - val_accuracy: 0.7677 - val_loss: 1.2785
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.9789 - loss: 0.0437 - val_accuracy: 0.7677 - val_loss: 1.3584


In [88]:
model_5_pred_prop = model_5.predict(val_sentences)
model_5_pred_prop = model_5_pred_prop.reshape(-1)
model_5_pred_prop[:10]

model_5_pred = tf.squeeze(tf.round(model_5_pred_prop))

model_5_pred[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [89]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int64)

In [90]:
calculate_accuracy(val_labels, model_5_pred)

{'model accuracy': 76.77165354330708,
 'model precision': 76.74723453090631,
 'model recall': 76.77165354330708,
 'model f1 score': 76.68863186407148}